In [1]:
import numpy as np
from obspy.core import UTCDateTime
from obspy import read
from obspy.taup import TauPyModel
from obspy.geodetics import locations2degrees
from glob import glob
import os
import os.path

In [2]:
project_name = 'newfault'
GF_list = 'rc_gflist.gflist' # Station file used in fq gen code
vel_mod = 'mojave'
stas_name = GF_list.split('.')[0]
fq_dir = '/hdd/rc_fq/fall24/' + project_name + '/'

ruptures = np.load('/hdd/rc_fq/fall24/' + project_name + '_ruptures.npy')
test_ruptures = ['newfault.000000', 'newfault.000001', 'newfault.000002']

# Where to save arrival times

arrival_save_dir = '/hdd/rc_fq/fall24/' + project_name + '_fq_parrivals/'
if os.path.isdir(arrival_save_dir):
    pass
else:
    os.makedirs(arrival_save_dir)

In [ ]:
# w = open('/home/sdybing/rc_fq/test_p_arrivals.csv', 'w') # saves arrivals for every event in one csv. Not using currently
counter = 0

for rupture in ruptures:
    
    counter += 1
    w = open(arrival_save_dir + stas_name + '_arrival_times_' + rupture + '.csv', 'w') # saves arrivals for each event in a separate csv
    
    print('Rupture ' + rupture + ' (' + str(counter) + '/' + str(len(ruptures)) + ')')
    
    log = glob(fq_dir + 'output/ruptures/' + rupture + '.log')
    f = open(log[0],'r')
    line = f.readlines()
    
    # getting hypocenter location
    hyp_loc_junk = line[16]
    hyp_loc_1 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[0])
    hyp_loc_2 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[1])
    hyp_loc_3 = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[2])
    
    epicenter = []
    epicenter.append(hyp_loc_1)
    epicenter.append(hyp_loc_2)
    epicenter.append(hyp_loc_3)
#     print(epicenter)
    
    epi_lon = epicenter[0]
    epi_lat = epicenter[1]
    epi_depth = epicenter[2]
    
    # getting hypocenter time
    hyp_time_junk = line[17]
    hyp_time = hyp_time_junk.split(' ')[2].split('Z')[0]
    time_epi = UTCDateTime(hyp_time)  
    stations = np.load('/hdd/rc_fq/fall24/' + stas_name + '_station_names.npy')
    Nsta = len(stations) # number of stations
    
    rootpath = fq_dir + '/output/waveforms/' + rupture + '/' # waveform file locations
    lonlat_load = np.load('/hdd/rc_fq/fall24/' + stas_name + '_station_namesandlocs.npy') 
    lonlat = lonlat_load[:,0:2] # station locations
    
    station_catalogue_load = np.load('/hdd/rc_fq/fall24/' + stas_name + '_station_namesandlocs.npy')
    station_catalogue = station_catalogue_load[:,2]
    
    velmod = TauPyModel(model = fq_dir + '/structure/' + vel_mod + '.npz') # velocity model
    
    predictedP = 9999 * np.ones(len(stations))
    
    # Get predicted arrivals
    
    for ksta in range(len(stations)):
        
        # Find station coordinates
        i = np.where(station_catalogue == stations[ksta])[0]
        lon_sta = float(lonlat[i,0])
        lat_sta = float(lonlat[i,1])
                        
        deg = locations2degrees(lat_sta, lon_sta, epi_lat, epi_lon) # calculates distance between station loc and epicenter loc
        arrivals = velmod.get_travel_times(source_depth_in_km = epi_depth, distance_in_degree = deg, phase_list = ['P','Pn','p'])
        
        # Determine P and S arrivals
        for kphase in range(len(arrivals)):
            if 'P' == arrivals[kphase].name or 'p' == arrivals[kphase].name or 'Pn' == arrivals[kphase].name:
                if arrivals[kphase].time < predictedP[ksta]:
                    predictedP[ksta] = arrivals[kphase].time
                    
#         print('Station ' + stations[ksta] + ': predicted arrival = ' + str(time_epi + predictedP[ksta]))
        
        line = '%s\t%s\t%s\n'%(rupture, str(stations[ksta]), str(time_epi + predictedP[ksta]))
        # print(line)
        w.write(line)
    
    w.close()

# w.close()



Rupture newfault.000000 (1/3300)
Rupture newfault.000001 (2/3300)
Rupture newfault.000002 (3/3300)
Rupture newfault.000003 (4/3300)
Rupture newfault.000004 (5/3300)
Rupture newfault.000005 (6/3300)
Rupture newfault.000006 (7/3300)
Rupture newfault.000007 (8/3300)
Rupture newfault.000008 (9/3300)
Rupture newfault.000009 (10/3300)
Rupture newfault.000010 (11/3300)
Rupture newfault.000011 (12/3300)
Rupture newfault.000012 (13/3300)
Rupture newfault.000013 (14/3300)
Rupture newfault.000014 (15/3300)
Rupture newfault.000015 (16/3300)
Rupture newfault.000016 (17/3300)
Rupture newfault.000017 (18/3300)
Rupture newfault.000018 (19/3300)
Rupture newfault.000019 (20/3300)
Rupture newfault.000020 (21/3300)
Rupture newfault.000021 (22/3300)
Rupture newfault.000022 (23/3300)
Rupture newfault.000023 (24/3300)
Rupture newfault.000024 (25/3300)
Rupture newfault.000025 (26/3300)
Rupture newfault.000026 (27/3300)
Rupture newfault.000027 (28/3300)
Rupture newfault.000028 (29/3300)
Rupture newfault.000029